In [1]:
from jupyter_dash import JupyterDash
import dash
from dash import dcc, html, Input, Output, dash_table
import dash_leaflet as dl
import plotly.express as px
import pandas as pd
import base64
from aac_shelter_sniles import AnimalShelter

username = "aacuser"
password = "NewPassword1234"
host = "nv-desktop-services.apporto.com"
port = 30373
database = "AAC"
collection = "animals"

shelter = AnimalShelter(username, password, host, port, database, collection)
df = pd.DataFrame.from_records(shelter.find_animals({}))

# Check if '_id' column exists before dropping it
if '_id' in df.columns:
    df.drop(columns=['_id'], inplace=True)

app = JupyterDash('AnimalShelterDashboard')

# Read the image and encode to Base64
image_path = 'Grazioso_Salvare_Logo.png'
encoded_image = base64.b64encode(open(image_path, 'rb').read()).decode('ascii')

# Define layout
app.layout = html.Div([
    html.Div([
        html.A([html.Img(src=f'data:image/png;base64,{encoded_image}', style={'width': '150px', 'float': 'left', 'margin-right': '10px'})], href='http://www.snhu.edu'),
        html.H1('Animal Shelter Dashboard by Stanley Niles', style={'textAlign': 'center'}),
    ]),
    html.Div([
        html.Button('Water Rescue', id='water-btn', n_clicks=0),
        html.Button('Mountain or Wilderness Rescue', id='mountain-btn', n_clicks=0),
        html.Button('Disaster or Individual Tracking', id='disaster-btn', n_clicks=0),
        html.Button('Reset', id='reset-btn', n_clicks=0),
    ], style={'margin-left': '0px', 'clear': 'both'}),
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i} for i in df.columns],
        data=df.to_dict('records'),
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        page_action="native",
        page_current=0,
        page_size=10,
        row_selectable='single',
        selected_rows=[0]
    ),
    html.Div([
        html.Div(id='map-id', style={'width': '50%', 'height': '300px', 'float': 'left'}),
        dcc.Graph(id='pie-chart', style={'width': '50%', 'height': '300px', 'float': 'right'})
    ])
])

# Callback for the button bar
@app.callback(
    [Output('datatable-id', 'data'),
     Output('datatable-id', 'selected_rows'),
     Output('pie-chart', 'figure')],
    [Input('water-btn', 'n_clicks'),
     Input('mountain-btn', 'n_clicks'),
     Input('disaster-btn', 'n_clicks'),
     Input('reset-btn', 'n_clicks')]
)
def update_data(water_clicks, mountain_clicks, disaster_clicks, reset_clicks):
    ctx = dash.callback_context
    button_id = ctx.triggered[0]['prop_id'].split('.')[0]

    # Filter based on the button clicked
    if button_id == 'water-btn':
        filtered_df = df[
            (df['breed'].isin(['Labrador Retriever Mix', 'Chesapeake Bay Retriever', 'Newfoundland'])) &
            (df['sex_upon_outcome'] == 'Intact Female') &
            (df['age_upon_outcome_in_weeks'] >= 26) &
            (df['age_upon_outcome_in_weeks'] <= 156)
        ]
    elif button_id == 'mountain-btn':
        filtered_df = df[
            (df['breed'].isin(['German Shepherd', 'Alaskan Malamute', 'Old English Sheepdog', 'Siberian Husky', 'Rottweiler'])) &
            (df['sex_upon_outcome'] == 'Intact Male') &
            (df['age_upon_outcome_in_weeks'] >= 26) &
            (df['age_upon_outcome_in_weeks'] <= 156)
        ]
    elif button_id == 'disaster-btn':
        filtered_df = df[
            (df['breed'].isin(['Doberman Pinscher', 'German Shepherd', 'Golden Retriever', 'Bloodhound', 'Rottweiler'])) &
            (df['sex_upon_outcome'] == 'Intact Male') &
            (df['age_upon_outcome_in_weeks'] >= 20) &
            (df['age_upon_outcome_in_weeks'] <= 300)
        ]
    else:
        filtered_df = df

    # Create a pie chart
    pie_df = filtered_df['breed'].value_counts().reset_index()
    pie_df.columns = ['breed', 'count']
    total_count = pie_df['count'].sum()
    pie_df['percentage'] = pie_df['count'] / total_count

    # Grouping breeds with less than 1% as "Other"
    other_percentage = pie_df[pie_df['percentage'] < 0.01]['percentage'].sum()
    pie_df = pie_df[pie_df['percentage'] >= 0.01]

    # Using pandas.concat instead of DataFrame.append
    other_row = pd.DataFrame({'breed': ['Other'], 'count': [0], 'percentage': [other_percentage]})
    pie_df = pd.concat([pie_df, other_row], ignore_index=True)

    # Plotting the pie chart based on the 'percentage' column
    figure = px.pie(pie_df, values='percentage', names='breed', title='Distribution by Breed')

    return filtered_df.to_dict('records'), [0], figure


@app.callback(
    Output('map-id', 'children'),
    [Input('datatable-id', 'selected_rows'),
     Input('datatable-id', 'data')]
)
def update_map(selected_rows, data):
    selected_row = data[selected_rows[0]]
    return dl.Map(style={'width': '100%', 'height': '300px'}, center=[selected_row['location_lat'], selected_row['location_long']], zoom=14, children=[
        dl.TileLayer(),
        dl.Marker(position=[selected_row['location_lat'], selected_row['location_long']], children=[
            dl.Tooltip(selected_row['name']),
            dl.Popup([
                html.H1('Animal Name: ' + selected_row['name']),
                html.P('Animal Type: ' + selected_row['animal_type'])
            ])
        ])
    ])

app.run_server(mode='inline', debug=False)


 * Running on http://127.0.0.1:10870/ (Press CTRL+C to quit)
127.0.0.1 - - [17/Aug/2023 21:58:26] "GET /_alive_b283f8a9-a4a8-46af-a4e3-9815096f8252 HTTP/1.1" 200 -


127.0.0.1 - - [17/Aug/2023 21:58:26] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [17/Aug/2023 21:58:26] "GET /_dash-component-suites/dash/deps/polyfill@7.v2_8_1m1675174511.12.1.min.js HTTP/1.1" 200 -
127.0.0.1 - - [17/Aug/2023 21:58:26] "GET /_dash-component-suites/dash/deps/react@16.v2_8_1m1675174511.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [17/Aug/2023 21:58:26] "GET /_dash-component-suites/dash/deps/prop-types@15.v2_8_1m1675174511.8.1.min.js HTTP/1.1" 200 -
127.0.0.1 - - [17/Aug/2023 21:58:26] "GET /_dash-component-suites/dash/deps/react-dom@16.v2_8_1m1675174511.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [17/Aug/2023 21:58:26] "GET /_dash-component-suites/dash_leaflet/dash_leaflet.v0_1_23m1636397671.min.js HTTP/1.1" 200 -
127.0.0.1 - - [17/Aug/2023 21:58:26] "GET /_dash-component-suites/dash/dash-renderer/build/dash_renderer.v2_8_1m1675174511.min.js HTTP/1.1" 200 -
127.0.0.1 - - [17/Aug/2023 21:58:26] "GET /_dash-component-suites/dash/dcc/dash_core_components.v2_8_0m1675174511.js HTTP/1.1"